In [1]:
import os
import pandas as pd

# Đường dẫn đến folder chứa các file CSV
list_folder_malware = ['./CSVS/SMSmalware-CSVs/SMSmalware/Biige', './CSVS/SMSmalware-CSVs/SMSmalware/Fakeinst', './CSVS/SMSmalware-CSVs/SMSmalware/FakeNotify',
                       './CSVS/SMSmalware-CSVs/SMSmalware/Mazarbot', './CSVS/SMSmalware-CSVs/SMSmalware/Jifake', './CSVS/SMSmalware-CSVs/SMSmalware/Nandrobox']
                     #   './CSVS/SMSmalware-CSVs/SMSmalware/Plankton', './CSVS/SMSmalware-CSVs/SMSmalware/Zsone']

list_folder_benign = ['./CSVS/Benign-CSVs/Benign/Benign2017']

# Tạo một DataFrame để chứa dữ liệu từ tất cả các file CSV
list_df = []
all_data = pd.DataFrame()
#Đọc từng file CSV và nối dữ liệu vào DataFrame chung
# Đọc benign
i = 0
for folder in list_folder_benign: 
     file_list =  os.listdir(folder)
     csv_files = [file for file in file_list if file.endswith('.csv')]
     for csv in csv_files: 
            if i < 150: 
               file_path = os.path.join(folder, csv)
               data = pd.read_csv(file_path, header=0)
               data.columns = data.columns.str.replace(' ', '')
               data = data.sort_values(by='Timestamp')
               data['Label'] = 1
               list_df.append(data)
               all_data = pd.concat([all_data, data], ignore_index=True)
               i = i + 1
# Đọc malware
for folder in list_folder_malware: 
     file_list =  os.listdir(folder)
     csv_files = [file for file in file_list if file.endswith('.csv')]
     for csv in csv_files: 
        file_path = os.path.join(folder, csv)
        data = pd.read_csv(file_path, header=0)
        data.columns = data.columns.str.replace(' ', '')
        data = data.sort_values(by='Timestamp')
        data['Label'] = 0
        list_df.append(data)
        all_data = pd.concat([all_data, data], ignore_index=True)




# Hiển thị DataFrame chứa dữ liệu từ tất cả các file CSV
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232187 entries, 0 to 232186
Data columns (total 85 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   FlowID                   232183 non-null  object 
 1   SourceIP                 232187 non-null  object 
 2   SourcePort               232187 non-null  float64
 3   DestinationIP            232187 non-null  object 
 4   DestinationPort          232187 non-null  float64
 5   Protocol                 232187 non-null  float64
 6   Timestamp                232187 non-null  object 
 7   FlowDuration             232187 non-null  float64
 8   TotalFwdPackets          232187 non-null  float64
 9   TotalBackwardPackets     232187 non-null  float64
 10  TotalLengthofFwdPackets  232187 non-null  float64
 11  TotalLengthofBwdPackets  232187 non-null  float64
 12  FwdPacketLengthMax       232187 non-null  float64
 13  FwdPacketLengthMin       232187 non-null  float64
 14  FwdP

In [2]:
all_data['Label'].value_counts()

Label
0    131617
1    100570
Name: count, dtype: int64

In [3]:
columns_drop=['FlowID', 'SourceIP', 'DestinationIP', 'SourcePort', 'DestinationPort', 'Timestamp',
                               'PacketLengthStd', 'CWEFlagCount', 'Down/UpRatio','FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 
                               'BwdAvgBytes/Bulk', 'BwdAvgPackets/Bulk', 'BwdAvgBulkRate', 'FwdURGFlags', 'BwdURGFlags', 
                               'RSTFlagCount', 'ECEFlagCount', 'BwdPSHFlags', 'FwdAvgBytes/Bulk']
# Tính ma trận tương quan
data = all_data.drop(columns=columns_drop)
corr_matrix = data.corr()
corr_matrix['Label'].sort_values(ascending=False)

               


Label                      1.000000
ACKFlagCount               0.021929
URGFlagCount               0.018869
Init_Win_bytes_backward    0.014388
FINFlagCount               0.009952
                             ...   
FwdIATStd                 -0.056582
FlowIATMax                -0.061562
FlowDuration              -0.063843
FwdIATTotal               -0.071185
FwdIATMax                 -0.074151
Name: Label, Length: 65, dtype: float64

In [3]:
# Function to divide each cell's value by 10 until the largest number in the column is <= 10
def divide_until_largest_less_than_10(df):
    for column in df.columns.tolist():
        contains_only_numbers = df[column].apply(lambda x: isinstance(x, (int, float))).all()
        if contains_only_numbers:
            largest_number = df[column].max()
            while largest_number > 10:
                df[column] /= 10
                largest_number = df[column].max()            
    return df

# Iterate over each DataFrame in the list
for df in list_df:
    df_copy = df.copy()  # Create a copy to avoid modifying the original DataFrame
    df_modified = divide_until_largest_less_than_10(df_copy)
    list_df.pop(0)
    list_df.append(df_modified)

In [5]:
def convert_ip_label(df):
    le_columns = ['SourceIP', 'DestinationIP']
    ip_list = list(df['SourceIP'].unique()) + list(df['DestinationIP'].unique())
    ip_set = list(set(ip_list))

    for column in le_columns:
        list_unique = list(df[column].unique())
        for val in list_unique:
            df.loc[df[column] == val, column] = ip_set.index(val)

    df['DestinationIP'] = df['DestinationIP'].astype(int)
    df['SourceIP'] = df['SourceIP'].astype(int)
    return df

In [7]:
import networkx as nx
import matplotlib.pyplot as plt
list_graph = []
list_label = []
for df in list_df:
    df = convert_ip_label(df)
    graph = nx.from_pandas_edgelist(df, 'SourceIP', 'DestinationIP',
                                     create_using=nx.MultiDiGraph(), edge_attr=df.drop(columns=columns_drop).columns.values.tolist())
    list_graph.append(graph)
    count_value = len(df['Label'].unique())
    if count_value == 1: 
        label = df['Label'].unique()[0]
    else: 
        label = 0
    list_label.append(label)
    


In [8]:
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

def graph_to_pyg_data(graph, label):
    # Khởi tạo ma trận B_in với tất cả các phần tử bằng 0
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    B_in = torch.zeros((num_nodes, num_edges), dtype=torch.float32)
    B_out = torch.zeros((num_nodes, num_edges), dtype=torch.float32)
    for i, node in enumerate(graph.nodes):
        for j, edge in enumerate(graph.edges):
            if node == edge[1]:
                B_in[i, j] = 1
            if node == edge[0]:
                B_out[i, j] = 1
    Y = torch.tensor(label, dtype=torch.long)
    X = torch.tensor([list(graph.edges[edge].values()) for edge in graph.edges], dtype=torch.float)
    return Data(B_in=B_in,B_out=B_out, X=X, Y=Y)

In [9]:
list_graph_labeled = list(zip(list_graph, list_label))
list_data = []
for graph, label in list_graph_labeled:
    list_data.append(graph_to_pyg_data(graph, label))

In [10]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(list_data, test_size=0.2, random_state=42)

In [11]:
from torch_geometric.loader import DataLoader
# from torch.utils.data import Dataset
# class MyDataset(Dataset):
#     def __init__(self, list_data):
#         self.list_data = list_data

#     def __len__(self):
#         return len(self.list_data)

#     def __getitem__(self, idx):
#         B_in, B_out, X, Y = self.list_data[idx]

#         return {B_in, B_out, X, Y}

batch_size = 1
# DataLoader cho tập dữ liệu đào tạo
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# DataLoader cho tập dữ liệu kiểm thử
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphNeuralNetwork, self).__init__()
        self.fc0 = nn.Linear(input_dim, hidden_dim)
        self.fc1 = nn.Linear(2*hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(3*hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(3*hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
    def forward(self, B_in, B_out, X):
        E_0 = F.relu(self.fc0(X))
        matrix1 = torch.matmul(B_in, E_0)
        matrix2 = torch.matmul(B_out, E_0)
        result_matrix_0 = torch.cat((matrix1, matrix2), dim=1)
        H_0 = F.relu(self.fc1(result_matrix_0))
        matrix3 = torch.matmul(B_in.t(), H_0)
        matrix4 = torch.matmul(B_out.t(), H_0)
        result_matrix_1 = torch.cat((matrix3, matrix4, E_0), dim=1)
        E_1 = F.relu(self.fc2(result_matrix_1))
        matrix5 = torch.matmul(B_in, E_1)
        matrix6 = torch.matmul(B_out, E_1)
        result_matrix_2 = torch.cat((matrix5, matrix6, H_0), dim=1)
        H_1 = F.relu(self.fc3(result_matrix_2))
        H_1_mean = torch.mean(H_1, dim=0, keepdim=True)
        # output = F.softmax(self.fc4(H_1_mean), dim=1)
        output = self.fc4(H_1_mean)
        return output

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Định nghĩa mô hình
input_dim =  65
hidden_dim =  64
output_dim =  2
model = GraphNeuralNetwork(input_dim, hidden_dim, output_dim)
print(model)
# Xác định hàm loss và trình tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
def train(model, train_loader, criterion, optimizer):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data.B_in, data.B_out, data.X)
        # print(f'---train---output: {output}\nY: {data.Y}')
        if torch.isnan(output).any():
            continue
        loss = criterion(output, torch.unsqueeze(data.Y, dim=0))
        # print(f'---train---loss: {loss}')
        loss.backward()
        optimizer.step()

# Đánh giá mô hình
def evaluate(model, loader):
    model.eval()
    
    correct = 0
    total_samples = 0
    for data in loader:
        output = model(data.B_in, data.B_out, data.X)
        print(f'---eval---output: {output}')
        pred = output.argmax(dim=1)
        print(f'---eval---pred: {pred}, Y: {data.Y}')
        correct += int((pred == data.Y).sum())
        total_samples += 1
    return correct / total_samples

# Huấn luyện và kiểm thử
num_epochs = 10
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion)
    test_accuracy = evaluate(model, test_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, Test Accuracy: {test_accuracy:.4f}')


GraphNeuralNetwork(
  (fc0): Linear(in_features=65, out_features=64, bias=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=192, out_features=64, bias=True)
  (fc3): Linear(in_features=192, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)


TypeError: 'Adam' object is not callable